In [6]:
from Data_Enrichment import get_features

RAW_DIR = "data/raw"
df_feats, feature_cols = get_features(RAW_DIR)
df_feats = df_feats[(df_feats['season_end_year'] != 2025) & (df_feats['season_end_year'] != 2026)]


/workspaces/Machine-learning/Data_Enrichment.py:232: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["ballon_dor_winner"] = df["ballon_dor_winner"].fillna(False).astype(bool)


In [16]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [8]:
# Target
y = df_feats['ballon_dor_winner'].values

# Features (excluyendo columnas no numéricas o irrelevantes)
X = df_feats.drop(columns=['ballon_dor_winner', 'player_name']).values


In [13]:
# Seleccionar solo columnas numéricas
X_numeric = df_feats.select_dtypes(include=['int64', 'float64',"bool"])


In [14]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numeric)


In [25]:
np.random.seed(30) #we fix the seed so we can reproduce the data any time
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_numeric, X_numeric["ballon_dor_winner"], test_size=0.3, random_state=42)

In [18]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Fit Lasso regression model
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

,alpha,0.1
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,1000
,tol,0.0001
,warm_start,False
,positive,False
,random_state,None
,selection,'cyclic'


In [19]:
y_pred = lasso.predict(X_test)
print(y_pred.mean())

0.00032230339492909325


In [30]:
# Model Score
print("Model Score: ", lasso.score(X_test, y_test))

Model Score:  -7.667844942327307e-06


/workspaces/Machine-learning/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


chat help me this part

In [31]:
import pandas as pd
import numpy as np

# Target
y = df_feats['ballon_dor_winner'].values  # 1 si ganó, 0 si no

# Features
# Excluimos target y player_name
X = df_feats.drop(columns=['ballon_dor_winner', 'player_name']).copy()

# Detectar columnas no numéricas
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
print("Categorical columns:", categorical_cols)

# Codificar variables categóricas
X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
print("Shape after encoding:", X_encoded.shape)

# Convertimos a numpy array
X_values = X_encoded.values


Categorical columns: ['season_name', 'team_name', 'competition_id', 'competition_name', 'position', 'main_position', 'foot']
Shape after encoding: (26595, 380)


In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_values, y, test_size=0.2, random_state=42, stratify=y
)


In [33]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [34]:
from sklearn.linear_model import LogisticRegression

log_lasso = LogisticRegression(
    penalty='l1',      # Lasso
    solver='saga',     # necesario para L1
    class_weight='balanced',  # ayuda con clases desbalanceadas
    max_iter=10000,
    random_state=42
)

# Entrenamos
log_lasso.fit(X_train_scaled, y_train)


,penalty,'l1'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,42
,solver,'saga'
,max_iter,10000
,multi_class,'deprecated'


In [35]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

# Predicciones de probabilidad
y_pred_prob = log_lasso.predict_proba(X_test_scaled)[:,1]

# Predicciones binarias con threshold 0.5
y_pred = (y_pred_prob >= 0.5).astype(int)

# Métricas
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred_prob))


Accuracy: 0.999059973679263
Precision: 0.2
Recall: 0.5
ROC AUC: 0.5615948843332705


In [36]:
# Obtener índices del test set
_, X_test_idx, _, _ = train_test_split(
    np.arange(len(df_feats)), y, test_size=0.2, random_state=42, stratify=y
)

# DataFrame con nombres y probabilidades
df_preds = df_feats.iloc[X_test_idx][['player_name']].copy()
df_preds['pred_prob'] = y_pred_prob

# Top 10 jugadores según modelo
top_players = df_preds.sort_values(by='pred_prob', ascending=False).head(10)
print(top_players)


                      player_name  pred_prob
11413        Lionel Messi (28003)   0.972810
11423        Lionel Messi (28003)   0.938812
11415        Lionel Messi (28003)   0.931769
11426        Lionel Messi (28003)   0.890241
19099         Luis Suárez (44352)   0.686258
26032              Neymar (68290)   0.327232
3932       Mohamed Salah (148455)   0.165768
16701  Robert Lewandowski (38253)   0.160709
460        Ciro Immobile (105521)   0.082597
3931       Mohamed Salah (148455)   0.047434


In [37]:
# Coeficientes del modelo
coef = log_lasso.coef_[0]
feature_names = X_encoded.columns

# Ordenar por importancia absoluta
feature_importance = pd.DataFrame({
    'feature': feature_names,
    'coef': coef
}).sort_values(by='coef', key=abs, ascending=False)

print(feature_importance.head(10))


                            feature      coef
19                          a_per90  0.717416
1                             goals  0.427976
45                   g_per90_z_lag1 -0.417175
14                        subed_out -0.399461
20                         ga_per90  0.397802
364  position_Attack - Right Winger  0.384170
62                        g_per90_w -0.332574
28                        g_per90_z -0.321814
61                       ga_per90_w -0.309869
73                season_name_06/07 -0.285024
